In [76]:
q=MXGSStartupHousekeepingTM(
 tcp_count_dpu                   = 1,
    acquisition_start_time_current  = 1,
    boot_status_completed_with_errors = 0,
    boot_status_completed_no_errors = 0,
    boot_status_asw_load_ok         = 0,
    boot_status_system_memory_test_ok = 0,
    boot_software_version_major     = 1,
    boot_software_version_minor     = 1,
    app_software_version_major      = 1,
    app_software_version_minor      = 1,
    app_software_version_patch      = 1
)

q.save()

In [102]:

import os, sys

home="/Users/garmu"
proj_path = home+"/Dropbox/asdc/"
# This is so Django knows where to find stuff.
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "asdc.settings")
sys.path.append(proj_path)

# This is so my local_settings.py gets loaded.
os.chdir(proj_path)

# This is so models get loaded.
from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


from asim.models import * 
    
def writedicttocdf(cdffilename,mytgfobs2,czt_det_counts  , bgo_det_counts):

    from spacepy import pycdf
    import os 

    pycdf.lib.set_backward(False)

    #cdffilename='/Users/garmu/Dropbox/notebooks/asimdata/testmxgsdata.cdf'
    try:
        os.remove(cdffilename) 
    except:
        print (cdffilename, "does not exist ")
    bad_keys=['_state', 'detector_counts','mxgs_trig_count']
    bad_keys=['_state', 'detector_counts']

    with pycdf.CDF(cdffilename, '') as cdf_file:    
        for key in sorted(mytgfobs2.__dict__.iterkeys()):
            if key not in bad_keys:
                value=mytgfobs2.__dict__[key]
                #print (key, value)
                cdf_file.new(key, value, recVary=False)
                #cdf_file.attrs[key] = value

        for key in sorted(czt_det_counts.iterkeys()):
            value=czt_det_counts[key]
            cdf_file.new(key, value, recVary=False)

        for key in sorted(bgo_det_counts.iterkeys()):
            value=bgo_det_counts[key]
            #print (key)
            cdf_file.new(key, value, recVary=False)
            #cdf_file.attrs['mxgs_trig_count'] = mytgfobs2.__dict__['mxgs_trig_count']
            #cdf_file.attrs[key] = value
    print ('wrote to file' , cdffilename)

def testfiletodict(fname):

    with open(fname, 'r') as fileData:
        TGFdict = {}
        tag = None
        tag2= None
        for line in fileData:
            #line = line.strip()
            #print (line)
            if not line:
                continue
            if line[0] == ' ':
                tag = line[1:].strip()
                tag2=tag.replace(' ','_')
                tag2=tag2.replace('-','_')
                tag2=tag2.lower()
                TGFdict[tag2] = ''
            else:
                assert tag is not None, 'Invalid format, does not have space at beginning of tag line'
                TGFdict[tag2] += line
    return TGFdict


def convert_detector_counts (TGFdict):
    '''
    trim the first four hex characters /zeros (16 bits)
    add 0x before?
    then convert to bitstring types and check whether from CZT or BGO
    then extract 12 bits pulse height
    20 bits timestamp

    '''

    detector_counts=TGFdict['detector_counts'].splitlines()

    print (detector_counts[1])

    from bitstring import BitArray, BitStream

    def remove_cruft(s):
        return BitArray('0x'+s[4:])


    det_counts=[remove_cruft(s) for s in detector_counts]

    czt_dau_address=[]
    czt_asic_address=[]
    czt_asic_channel=[]
    czt_pulse_height=[]
    czt_multi_hit=[]
    czt_timestamp=[]

    czt_det_counts = {'czt_dau_address': czt_dau_address, \
                      'czt_asic_address': czt_asic_address, \
                      'czt_asic_channel': czt_asic_channel, \
                      'czt_pulse_height': czt_pulse_height, \
                      'czt_multi_hit': czt_multi_hit, \
                      'czt_timestamp': czt_timestamp     }



    bgo_address=[]
    bgo_fast=[]
    bgo_ovf=[]
    bgo_valley=[]
    bgo_pulse_height=[]
    bgo_timestamp=[]
    bgo_fast_timestamp=[]

    bgo_det_counts = {'bgo_address': bgo_address, \
                      'bgo_ovf': bgo_ovf, \
                      'bgo_fast': bgo_fast, \
                      'bgo_valley': bgo_valley, \
                      'bgo_pulse_height': bgo_pulse_height, \
                      'bgo_timestamp': bgo_timestamp, \
                      'bgo_fast_timestamp': bgo_fast_timestamp     }

    for i in range (0,len(det_counts)):
        a=det_counts[i]
        #print (a[0:2])
        if (a[0:2]== '0b00') :
            flagbits= a[0:2].uint
            dau_address=a[2:4].uint
            asic_address = a[4:9].uint
            asic_channel = a[9:16].uint
            pulse_height = a[16:26].uint
            multi_hit = a[26:28].uint
            timestamp = a[28:48].uint  

            czt_det_counts['czt_dau_address'].append(dau_address)
            czt_det_counts['czt_asic_address'].append(asic_address)
            czt_det_counts['czt_asic_channel'].append(asic_channel)
            czt_det_counts['czt_pulse_height'].append(pulse_height)
            czt_det_counts['czt_multi_hit'].append(multi_hit)
            czt_det_counts['czt_timestamp'].append(timestamp)
            """x
            print 'CZT' ,flagbits  ,\
                   'dau_add' ,dau_address, \
                   'asic_add' ,asic_address, \
                   'asic_chan' ,asic_channel, \
                   'puls_ht',pulse_height,\
                   'multi_hit', multi_hit, \
                   'time', timestamp
            x"""
            #if (multi_hit > 0):
            #    print ('MULTI_HIT!!!')
        if (a[0:2]== '0b10') :
            flagbits= a[0:2].uint
            address = a[2:6].uint        
            ovf = a[6]        
            fast = a[7]
            valley=a[8]
            bgostr= 'BGO, '+str (flagbits)+\
                   ' address, '+str(address)+ \
                   ' ovf, '+str(ovf)+ \
                   ' fast,'+str(fast)
            if ( ovf == 0 ):
                spare=a[9]
                fast_timestamp=a[10:16].uint
                pulse_height=a[16:28].uint
                timestamp=a[28:48].uint

                bgo_det_counts['bgo_address'].append(address)
                bgo_det_counts['bgo_ovf'].append(ovf)
                bgo_det_counts['bgo_fast'].append(fast)
                bgo_det_counts['bgo_valley'].append(valley)
                bgo_det_counts['bgo_pulse_height'].append(pulse_height)
                bgo_det_counts['bgo_timestamp'].append(timestamp)
                bgo_det_counts['bgo_fast_timestamp'].append(fast_timestamp)
                #print bgostr, 'fast_timestamp', fast_timestamp, 'pulse_height', pulse_height,'timestamp', timestamp
            if ( ovf == 1 and fast==0):
                spare=a[9:16].uint
                overflow_duration=a[16:28].uint
                timestamp=a[28:48].uint
                #print bgostr, 'overflow, ', overflow_duration, 'timestamp, ', timestamp
            if ( ovf == 1 and fast==1):
                sample=a[9:16].uint
                adc_value=a[16:28].uint
                timestamp=a[28:48].uint
                #print bgostr, 'sample, ', sample,'adc_value, ', adc_value, 'timestamp', timestamp
    
    return czt_det_counts, bgo_det_counts



def parse_tgf_data(TGFdict):
    trigger_window_thresholds=[int(s) for s in TGFdict['trigger_window_thresholds'].split(' ')]
    trigger_window_durations=[int(s) for s in TGFdict['trigger_window_durations'].split(' ')]
    dau_1_internal_tmons=[int(s) for s in TGFdict['dau_1_internal_tmons'].split(' ')]
    dau_2_internal_tmons=[int(s) for s in TGFdict['dau_2_internal_tmons'].split(' ')]
    dau_3_internal_tmons=[int(s) for s in TGFdict['dau_3_internal_tmons'].split(' ')]
    dau_4_internal_tmons=[int(s) for s in TGFdict['dau_4_internal_tmons'].split(' ')]
    time_of_latest_tcp=[int(s) for s in TGFdict['time_of_latest_tcp'].split(' ')]
    accepted_count_parameters=[int(s) for s in TGFdict['accepted_count_parameters'].split(' ')]
    mmia_triggering_time=[int(s) for s in TGFdict['mmia_triggering_time'].split(' ')]

    mxgs_triggering_count=long(TGFdict['mxgs_triggering_count'],0)


    observation_time=[int(s) for s in TGFdict['observation_time'].split(' ')]


    import bitarray
    ba = bitarray.bitarray()
    ba.fromstring(TGFdict['trigger_window_flags'])
    trigger_window_flags = ba.tolist()

    #print ('mxgs_triggering_count', mxgs_triggering_count)



    #detector_counts=[int(s) for s in TGFdict['detector_counts'].split('\n')]

    detector_counts=TGFdict['detector_counts'].splitlines()


    mytgfobs2=MXGSTGFObservation(
    observation_id                  =int (TGFdict['observation_id'],16),
    utc_year                        =time_of_latest_tcp[0],
    utc_seconds                     =time_of_latest_tcp[1],
    utc_msec                        =time_of_latest_tcp[2],
    tcp_count_dhpu                  =observation_time[0], 
    tcp_count_dpu                   =observation_time[1], 
    dpu_count                       =observation_time[2], 
    dpu_count_prereset              =int (TGFdict['dpu_counter_pre_reset']), 
    dau_bgo_1_int_tmon_chan1        =dau_1_internal_tmons[0], 
    dau_bgo_1_int_tmon_chan2        =dau_1_internal_tmons[1], 
    dau_bgo_1_int_tmon_chan3        =dau_1_internal_tmons[2], 
    dau_bgo_1_int_tmon_chan4        =dau_1_internal_tmons[3],  
    dau_bgo_2_int_tmon_chan1        =dau_2_internal_tmons[0], 
    dau_bgo_2_int_tmon_chan2        =dau_2_internal_tmons[1],
    dau_bgo_2_int_tmon_chan3        =dau_2_internal_tmons[2],
    dau_bgo_2_int_tmon_chan4        =dau_2_internal_tmons[3], 
    dau_bgo_3_int_tmon_chan1        =dau_3_internal_tmons[0], 
    dau_bgo_3_int_tmon_chan2        =dau_3_internal_tmons[1], 
    dau_bgo_3_int_tmon_chan3        =dau_3_internal_tmons[2],  
    dau_bgo_3_int_tmon_chan4        =dau_3_internal_tmons[3], 
    dau_bgo_4_int_tmon_chan1        =dau_4_internal_tmons[0],  
    dau_bgo_4_int_tmon_chan2        =dau_4_internal_tmons[1],  
    dau_bgo_4_int_tmon_chan3        =dau_4_internal_tmons[2],  
    dau_bgo_4_int_tmon_chan4        =dau_4_internal_tmons[3],   
    led_short_win_lr_pulse_height   =accepted_count_parameters[0], 
    led_short_win_upr_pulse_height  =accepted_count_parameters[1], 
    led_long_win_lr_pulse_height    =accepted_count_parameters[2], 
    led_long_win_upr_pulse_height   =accepted_count_parameters[3], 
    hed_short_win_lr_pulse_height   =accepted_count_parameters[4],  
    hed_short_win_upr_pulse_height  =accepted_count_parameters[5], 
    hed_long_win_lr_pulse_height    =accepted_count_parameters[6], 
    hed_long_win_upr_pulse_height   =accepted_count_parameters[7], 
    led_short_win_anticoin_time     =accepted_count_parameters[8], 
    led_long_win_anticoin_time      =accepted_count_parameters[9], 
    hed_short_win_anticoin_time     =accepted_count_parameters[10],  
    hed_long_win_anticoin_time      =accepted_count_parameters[11],  
    led_short_win_flag1             =trigger_window_flags[0],
    led_short_win_flag2             =trigger_window_flags[1],
    led_short_win_flag3             =trigger_window_flags[2],
    led_long_win_flag               =trigger_window_flags[3],
    hed_short_win_flag1             =trigger_window_flags[4],
    hed_short_win_flag2             =trigger_window_flags[5],
    hed_short_win_flag3             =trigger_window_flags[6],
    hed_long_win_flag               =trigger_window_flags[7],
    trig_mmia_enabled               =mmia_triggering_time[0],
    trig_mmia_recd                  =mmia_triggering_time[1],
    led_short_win_dur_1             =trigger_window_durations[0], 
    led_short_win_dur_2             =trigger_window_durations[1], 
    led_short_win_dur_3             =trigger_window_durations[2], 
    led_long_win_dur                =trigger_window_durations[3], 
    hed_short_win_dur_1             =trigger_window_durations[4], 
    hed_short_win_dur_2             =trigger_window_durations[5], 
    hed_short_win_dur_3             =trigger_window_durations[6], 
    hed_long_win_dur                =trigger_window_durations[7], 
    led_short_win_thresh_1          =trigger_window_thresholds[0], 
    led_short_win_thresh_2          =trigger_window_thresholds[1], 
    led_short_win_thresh_3          =trigger_window_thresholds[2], 
    led_long_win_thresh             =trigger_window_thresholds[3], 
    hed_short_win_thresh_1          =trigger_window_thresholds[4], 
    hed_short_win_thresh_2          =trigger_window_thresholds[5], 
    hed_short_win_thresh_3          =trigger_window_thresholds[6], 
    hed_long_win_thresh             =trigger_window_thresholds[7], 
    mxgs_trig_count                 =mxgs_triggering_count,
    mxgs_trig_tcp_count             =8, 
    mxgs_trig_dpu_count             =8, 
    num_counts                      =int (TGFdict['number_of_counts']), 
    detector_counts                 ='asimdata/2016/117/mydata.cdf'  
    )
    mytgfobs2.save()

    czt_det_counts, bgo_det_counts=convert_detector_counts (TGFdict)
    
    debug=0
    if debug==1:
        for key in sorted(mytgfobs2.__dict__.iterkeys()):
            if key not in bad_keys:
                value=mytgfobs2.__dict__[key]
                print (key, value)

    print (mytgfobs2)

    return mytgfobs2, czt_det_counts, bgo_det_counts


def mxgstestdata(fname):
    TGFdict = testfiletodict(fname)

    # read in a file from fname, add the keys to a dict and loop through multiline data fields
    

    mytgfobs2, czt_det_counts, bgo_det_counts=parse_tgf_data(TGFdict)


    
    cdffilename=fname.replace(".txt ",".cdf")
    cdffilename=os.path.basename(fname).replace(".txt",".cdf")
    print (cdffilename)
    writedicttocdf(cdffilename,mytgfobs2,czt_det_counts  , bgo_det_counts)
    print ("done")
    
    return 0
    


path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/TGF/'





fnames=os.listdir(path) 

#fnames=fnamelist.split()
q=MXGSTGFObservation.objects.all().delete()

print fnames
for fname in fnames:    
    mxgstestdata(path+fname)


['Obs_0005.txt', 'Obs_000c.txt', 'Obs_0018.txt', 'Obs_001b.txt', 'Obs_001e.txt', 'Obs_0024.txt', 'Obs_0027.txt', 'Obs_002a.txt', 'Obs_002f.txt']
00000fcc200a565c
obsid= 5, mxgs_trig_count= 162826650543631, led_short_win_dur_1= 300, num_counts= 8798 
Obs_0005.cdf
('wrote to file', 'Obs_0005.cdf')
done
0000941c0c09d527
obsid= 12, mxgs_trig_count= 140759069742937, led_short_win_dur_1= 300, num_counts= 7504 
Obs_000c.cdf
('wrote to file', 'Obs_000c.cdf')
done
00000fcc1d864141
obsid= 24, mxgs_trig_count= 162792309409988, led_short_win_dur_1= 300, num_counts= 8074 
Obs_0018.cdf
('wrote to file', 'Obs_0018.cdf')
done
00000fcc2005296f
obsid= 27, mxgs_trig_count= 36968683883777, led_short_win_dur_1= 300, num_counts= 7562 
Obs_001b.cdf
('wrote to file', 'Obs_001b.cdf')
done
00001e24da88d034
obsid= 30, mxgs_trig_count= 158420161849455, led_short_win_dur_1= 300, num_counts= 7190 
Obs_001e.cdf
('wrote to file', 'Obs_001e.cdf')
done
00000fcc230e44d1
obsid= 36, mxgs_trig_count= 193643002357163, led_s

In [106]:


def parseMXGSSampled (fname):
    listofdicts=[]
    sampd={}
    sampbeg={}

    with open (fname) as f:
        for line in f:
            if (line[0] == ' '):
                data=next(f)
                #print (next(f))
                tag2=line.strip()
                tag2=tag2.replace(' ','_')
                tag2=tag2.replace('-','_')
                tag2=tag2.lower()
                sampd[tag2]=data
                #print (line)
            if (line.find('Time of Latest TCP') != -1):
                # add sample to the list
                #print len(listofdicts)print ('how many times does this get called')
                listofdicts.append(sampd)
                if ( len(listofdicts) == 1):
                    dau_1_internal_tmons=[int(s) for s in sampd['dau_1_internal_tmons'].split(' ')]
                    dau_2_internal_tmons=[int(s) for s in sampd['dau_2_internal_tmons'].split(' ')]
                    dau_3_internal_tmons=[int(s) for s in sampd['dau_3_internal_tmons'].split(' ')]
                    dau_4_internal_tmons=[int(s) for s in sampd['dau_4_internal_tmons'].split(' ')]
                    samp=MXGSSampledDetectorCounts(
                        dau_bgo_1_int_tmon_chan1        =dau_1_internal_tmons[0],
                        dau_bgo_1_int_tmon_chan2        =dau_1_internal_tmons[1],
                        dau_bgo_1_int_tmon_chan3        =dau_1_internal_tmons[2],
                        dau_bgo_1_int_tmon_chan4        =dau_1_internal_tmons[3],
                        dau_bgo_2_int_tmon_chan1        =dau_2_internal_tmons[0],
                        dau_bgo_2_int_tmon_chan2        =dau_2_internal_tmons[1],
                        dau_bgo_2_int_tmon_chan3        =dau_2_internal_tmons[2],
                        dau_bgo_2_int_tmon_chan4        =dau_2_internal_tmons[3],
                        dau_bgo_3_int_tmon_chan1        =dau_3_internal_tmons[0],
                        dau_bgo_3_int_tmon_chan2        =dau_3_internal_tmons[1],
                        dau_bgo_3_int_tmon_chan3        =dau_3_internal_tmons[2],
                        dau_bgo_3_int_tmon_chan4        =dau_3_internal_tmons[3],
                        dau_bgo_4_int_tmon_chan1        =dau_4_internal_tmons[0],
                        dau_bgo_4_int_tmon_chan2        =dau_4_internal_tmons[1],
                        dau_bgo_4_int_tmon_chan3        =dau_4_internal_tmons[2],
                        dau_bgo_4_int_tmon_chan4        =dau_4_internal_tmons[3])
                    samp.save()
                if ( len(listofdicts) > 1):
                    timetcp=[int(s) for s in sampd['time_of_latest_tcp'].split(' ')]
                    obstime=[int(s) for s in sampd['observation_time'].split(' ')]
                    data_reduc=[int(s) for s in sampd['dau_data_reduction_factors'].split(' ')]
                    #print sampd
                    samp1sec=MXGSSampledDetectorCounts1Second(
                        mxgs_sampled_detector_counts = samp,
                        utc_year                        =timetcp[0],
                        utc_msec                        =timetcp[1],
                        utc_seconds                     =timetcp[2],
                        tcp_count_dhpu                  =obstime[0],
                        tcp_count_dpu                   =obstime[1],
                        dpu_count_prereset              =int(sampd['dpu_counter_pre_reset']),
                        dpu_count_Sample_Ratio          =data_reduc[0],
                        grey_mode_data_reduc_factor     =data_reduc[1],
                        detector_counts                 =int(sampd['number_of_counts']),
                        detector_events                 ='fixmedirectory/mysamp.cdf')
                    samp1sec.save()
                sampd={}


            if (line.find('Detector Counts') != -1):
                arr=[]
                n=int(sampd['number_of_counts'])
                for i in range (0, n-1):
                    arr.append(next(f))
                sampd['detector_counts']=arr
    return listofdicts



path='/Users/garmu/MXGS_PFM_data/PFM_20160513_0912/Section_5_3/Sampled/'


fnamelist=os.listdir(path) 

print (fnamelist)
#fnamelist="Obs_0007.txt"

#fname=path+fnamelist

q=MXGSSampledDetectorCounts.objects.all().delete()
q=MXGSSampledDetectorCounts1Second.objects.all().delete()
for fname in fnamelist:    
    parseMXGSSampled(path+fname)

['Obs_0007.txt', 'Obs_0009.txt', 'Obs_000b.txt', 'Obs_000e.txt', 'Obs_0011.txt', 'Obs_0015.txt', 'Obs_0017.txt', 'Obs_001a.txt', 'Obs_001d.txt', 'Obs_0020.txt', 'Obs_0023.txt', 'Obs_0026.txt', 'Obs_0029.txt', 'Obs_002c.txt', 'Obs_002e.txt', 'Obs_0032.txt', 'Obs_0036.txt']


In [103]:
print sampd

{'number_of_counts': '44\n', 'end': '00000000\n', 'detector_counts': ['00000fcc2180a86d\n', '000020e18f40f68e\n', '00000fcc1e8151a0\n', '00003ab018419bf8\n', '0000b4110201f8ac\n', '0000b80501b25af4\n', '00000fcc2182b010\n', '000084040112f867\n', '00000fcc20035931\n', '00000fcc20c3a824\n', '00000fcc21040750\n', '0000b40901a46442\n', '0000a0130544b929\n', '00000fcc1c850964\n', '000012f32ec5616b\n', '0000980b0305c4b4\n', '00000fcc1d062a20\n', '0000a81e05c67a30\n', '0000b81c0396e509\n', '00000fcc1f073abc\n', '00000fcc22079ba9\n', '00000fcc1e080d7e\n', '0000a8030338657a\n', '000031164c48c82f\n', '0000900001091633\n', '0000b41902e97202\n', '0000b00d08f9bdf0\n', '000010dc12ca1f41\n', '00000fcc1eca7976\n', '00001e4badcabd5b\n', '0000880f050b1055\n', '0000a80069eb6040\n', '00000fcc200bb122\n', '00000fcc210bf44b\n', '0000a423019c5474\n', '00000fcc1ecca632\n', '00009809028d0288\n', '000012962c4d62b7\n', '0000880d014dc4e7\n', '0000357e70ce23f9\n', '00000fcc204e7f0b\n', '00000fcc1f4ed809\n', '00000

In [ ]:
print 